In [20]:
system_content = 'You are the story evaluator. You just have to look at Story A and Story B, and answer the questions only with "A" or "B".'
print(system_content)

You are the story evaluator. You just have to look at Story A and Story B, and answer the questions only with "A" or "B".


In [53]:
import csv

selection_list_path = 'results/evaluation/selection_list.csv'
column_a = 'gpt_hu10'
column_b = 'ss_1'
goal = 'Kate ran in her new shoes. (22)' #@param {type:'string'}

with open(selection_list_path, mode='r') as file:
    reader = csv.DictReader(file)

    for row in reader:
        if column_a in row and column_b in row and row['goal'] == goal:
            skeleton_a = row[column_a].split('\n')
            skeleton_b = row[column_b].split('\n')
            break

In [54]:
question = 'Which story was more interesting?'

In [56]:
question = 'Which story had coherent flow between sentences?'

In [58]:
question = 'Which story had overall consistency in theme?'

In [59]:
prompt = f"""Story A"""
for i, s in enumerate(skeleton_a):
    prompt += f"""\n{i+1}. {s}"""
prompt += f"""\n\nStory B"""
for i, s in enumerate(skeleton_b):
    prompt += f"""\n{i+1}. {s}"""
prompt += f"""\n\nQuestion: {question}\nAnswer: """
print(prompt)

Story A
1. her own car was being repaired
2. she didn't have any other means of transportation
3. Kate walked to the place where she looked up the distance on a map
4. the location was within walking distance
5. Kate walked to the location where she flagged down the taxi
6. Kate flagged down a passing taxi
7. Kate took a taxi to get to where her friend is
8. Kate asked her friend to borrow a car
9. Kate bought new shoes
10. Kate ran in her new shoes.

Story B
1. Kate asked her friend to borrow a map
2. Kate borrowed a map from a friend
3. Kate walked to the place where she used the navigation app on her phone as a walking guide
4. Kate used the navigation app to ensure she took the correct route
5. Kate walked to the place where she looked up the distance on a map
6. Kate walked to the location where she flagged down the taxi
7. Kate flagged down a passing taxi
8. Kate asked her friend to borrow a car
9. Kate borrowed a car from a friend
10. Kate ran in her new shoes.

Question: Which 

# Set-up

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/2302_happyTEXT

/content/drive/MyDrive/2302_happyTEXT


In [ ]:
import csv

import nltk
nltk.download('wordnet')
nltk.download('tagsets')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tag  import pos_tag

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

from tqdm import tqdm
import pickle

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package tagsets to /root/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Load train dataset

In [ ]:
story_dataset = []

In [ ]:
# CSV 파일 경로
DATASET_PATH = 'ROCStories_.csv'

# CSV 파일 열기
with open(DATASET_PATH, mode='r') as file:
    csv_reader = csv.reader(file)

    # 각 행에 대해서 처리
    for idx, row in enumerate(csv_reader):
        if idx == 0:
            continue

        # 3-7 열에 스토리
        story = ''
        story += ' '.join(row[2:6]) + ' '
        story = story.strip()
        story_dataset.append(story)

In [ ]:
len(story_dataset)

52665

In [ ]:
story_dataset[:10]

["David noticed he had put on a lot of weight recently. He examined his habits to try and figure out the reason. He realized he'd been eating too much fast food lately. He stopped going to burger places and started a vegetarian diet.",
 "Tom had a very short temper. One day a guest made him very angry. He punched a hole in the wall of his house. Tom's guest became afraid and left quickly.",
 'Marcus needed clothing for a business casual event. All of his clothes were either too formal or too casual. He decided to buy a pair of khakis. The pair he bought fit him perfectly.',
 'Bobby thought Bill should buy a trailer and haul it with his car. Bill thought a truck would be better for what he needed. Bobby pointed out two vehicles were much more expensive. Bill was set in his ways with conventional thinking.',
 'John was a pastor with a very bad memory. He tried to memorize his sermons many days in advance but to no avail. He decided to learn to sing to overcome his handicap. He then made 

# Preprocessing


https://wikidocs.net/182553

In [ ]:
# tokeniziong
tokenizer = WordPunctTokenizer()

story_tokens = []
for story in story_dataset:
    story_tokens.append(tokenizer.tokenize(story))

In [ ]:
story_tokens[0]

['David',
 'noticed',
 'he',
 'had',
 'put',
 'on',
 'a',
 'lot',
 'of',
 'weight',
 'recently',
 '.',
 'He',
 'examined',
 'his',
 'habits',
 'to',
 'try',
 'and',
 'figure',
 'out',
 'the',
 'reason',
 '.',
 'He',
 'realized',
 'he',
 "'",
 'd',
 'been',
 'eating',
 'too',
 'much',
 'fast',
 'food',
 'lately',
 '.',
 'He',
 'stopped',
 'going',
 'to',
 'burger',
 'places',
 'and',
 'started',
 'a',
 'vegetarian',
 'diet',
 '.']

https://ysyblog.tistory.com/88

In [ ]:
def get_wordnet_pos(tag):
    if tag.startswith('V'):
        return 'v'
    elif tag.startswith('N'):
        return 'n'
    elif tag.startswith('J'):
        return 'a'
    elif tag.startswith('R'):
        return 'r'
    else:
        return None

In [ ]:
# 동사만 추출 (원형으로)
lemm = WordNetLemmatizer()
story_verb_tokens = []

for story in tqdm(story_tokens):
    verb_tokens = []
    for token, tag in pos_tag(story):
        if get_wordnet_pos(tag) == 'v':
            verb_tokens.append(lemm.lemmatize(token, pos=get_wordnet_pos(tag)))
    story_verb_tokens.append(verb_tokens)

100%|██████████| 52665/52665 [01:08<00:00, 770.21it/s]


In [ ]:
story_verb_tokens[:10]

[['notice',
  'have',
  'put',
  'examine',
  'try',
  'figure',
  'realize',
  'be',
  'eat',
  'stop',
  'go',
  'burger',
  'start'],
 ['have', 'make', 'punch', 'become', 'leave'],
 ['need', 'be', 'decide', 'buy', 'buy'],
 ['think', 'buy', 'haul', 'think', 'be', 'need', 'point', 'be', 'be', 'set'],
 ['be',
  'try',
  'memorize',
  'decide',
  'learn',
  'sing',
  'overcome',
  'make',
  'sing'],
 ['surprise', 'take', 'arrive', 'be', 'saw'],
 ['announce',
  'begin',
  'complain',
  'begin',
  'pass',
  'take',
  'begin',
  'work',
  'finish'],
 ['meet', 'live', 'meet', 'stay'],
 ['get', 'be', 's', 'sleep', 'be', 'lay', 'lay', 'smell'],
 ['love', 'decide', 'grow', 'grow', 'make', 'realize', 'require']]

In [ ]:
# 동사 토큰 이어서 하나의 string으로, tf-idf를 위해
story_verb = []
for tokens in story_verb_tokens:
    story_verb.append(' '.join(tokens))

In [ ]:
story_verb[:10]

['notice have put examine try figure realize be eat stop go burger start',
 'have make punch become leave',
 'need be decide buy buy',
 'think buy haul think be need point be be set',
 'be try memorize decide learn sing overcome make sing',
 'surprise take arrive be saw',
 'announce begin complain begin pass take begin work finish',
 'meet live meet stay',
 'get be s sleep be lay lay smell',
 'love decide grow grow make realize require']

# Train

In [ ]:
# TfidfVectorizer 객체 생성
vectorizer = TfidfVectorizer()

# vectorizer는 X_train에 등장하는 단어를 통해 단어 사전을 생성한 뒤,
# 해당 단어 사전을 사용해 X_train를 TfidfVectorizer 한다.
# 따라서, vec_X_train의 차원은 (X_train 문장 수, 사전의 단어 수) 가 된다.
vec_X_train = vectorizer.fit_transform(story_verb)

In [ ]:
# 벡터라이저가 학습한 단어사전 정렬
word_list = [i[0] for i in sorted(vectorizer.vocabulary_.items())]

# vec_X_train을 DataFrame 화
pd.DataFrame(vec_X_train.toarray(), columns=[word_list])

,aa,abandon,abduct,abide,ablaze,aboard,abolish,abort,absent,absinthe,...,youtube,yowl,zaddyfrack,zaddyfracks,zap,zigzag,zip,ziplining,zone,zoom
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# save
with open(f'idf_train_ROC_verb_TfidfVectorizer.pickle', 'wb') as f:
        pickle.dump(vec_X_train, f)